In [61]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

# Load environment variables
load_dotenv()

# Set up OpenAI API key
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [62]:
from litellm import completion


In [153]:
from langchain_community.chat_models import ChatLiteLLM
from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
    PromptTemplate
)


In [64]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage


In [65]:
# chat = ChatLiteLLM(model="gpt-4o-mini").bind(logprobs=True) 
# This does not work



In [66]:
chat = ChatOpenAI(model="gpt-4o-mini").bind(logprobs=True)
# Only OpenAI works for logprobs

In [67]:
chat


RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000002E43E695B50>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000002E43E4B6D20>, root_client=<openai.OpenAI object at 0x000002E43E687170>, root_async_client=<openai.AsyncOpenAI object at 0x000002E43E695B80>, model_name='gpt-4o-mini', model_kwargs={}, openai_api_key=SecretStr('**********')), kwargs={'logprobs': True}, config={}, config_factories=[])

In [68]:
messages = [
    HumanMessage(
        content="what model are you"
    )
]

In [69]:
msg = chat.invoke(messages)


In [70]:
msg.response_metadata["logprobs"]["content"][:5]


[{'token': 'I', 'bytes': [73], 'logprob': -0.038042292, 'top_logprobs': []},
 {'token': ' am',
  'bytes': [32, 97, 109],
  'logprob': -0.038042065,
  'top_logprobs': []},
 {'token': ' based',
  'bytes': [32, 98, 97, 115, 101, 100],
  'logprob': -0.000698496,
  'top_logprobs': []},
 {'token': ' on', 'bytes': [32, 111, 110], 'logprob': 0.0, 'top_logprobs': []},
 {'token': ' Open',
  'bytes': [32, 79, 112, 101, 110],
  'logprob': -0.014165984,
  'top_logprobs': []}]

In [71]:
from typing_extensions import Annotated, TypedDict
from typing import Optional

In [72]:
# # TypedDict
# class Joke(TypedDict):
#     """Joke to tell user."""

#     setup: Annotated[str, ..., "The setup of the joke"]

#     # Alternatively, we could have specified setup as:

#     # setup: str                    # no default, no description
#     # setup: Annotated[str, ...]    # no default, no description
#     # setup: Annotated[str, "foo"]  # default, no description

#     punchline: Annotated[str, ..., "The punchline of the joke"]
#     rating: Annotated[Optional[int], None, "How funny the joke is, from 1 to 10"]

In [73]:
# structured_chat = chat.with_structured_output(Joke)
# response = structured_chat.invoke(messages)

In [74]:
# response # If I force the llm to return a structured output, it does not return the logprobs


{'setup': 'What kind of model am I?',
 'punchline': "I'm an AI model, but I still can't find my way around a runway!",
 'rating': 6}

In [76]:
msg


AIMessage(content="I am based on OpenAI's GPT-3 model. My training includes a wide range of information up until October 2021, and I can assist with various topics, answer questions, and engage in conversation. How can I help you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 11, 'total_tokens': 61, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_39a40c96a0', 'finish_reason': 'stop', 'logprobs': {'content': [{'token': 'I', 'bytes': [73], 'logprob': -0.038042292, 'top_logprobs': []}, {'token': ' am', 'bytes': [32, 97, 109], 'logprob': -0.038042065, 'top_logprobs': []}, {'token': ' based', 'bytes': [32, 98, 97, 115, 101, 100], 'logprob': -0.000698496, 'top_logprobs': []}, {'token': ' on', 'byt

In [89]:
from langchain_community.document_loaders import DirectoryLoader


In [102]:
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_core.documents import Document

In [247]:
markdown_path = "C:\\Users\\DNSOFT\\test\\system_prompts\\"

# Get a list of all .md files in the directory
md_files = [f for f in os.listdir(markdown_path) if f.endswith('.md')]


In [248]:
# # Load each Markdown file using UnstructuredMarkdownLoader
# data = []
# for md_file in md_files:
#     loader = UnstructuredMarkdownLoader(os.path.join(markdown_path, md_file))  # Load each file
#     data.append(loader.load())  # Append the loaded data

In [249]:
# Load each Markdown file manually
data = []
for md_file in md_files:
    with open(os.path.join(markdown_path, md_file), 'r', encoding='utf-8') as file:  # Open each file
        content = file.read()  # Read the content of the file
        # Append a dictionary with name and content to the data list
        data.append({
            "name": md_file,  # Use the filename as the name
            "content": content  # Store the content
        })

In [250]:
data[0]['name'][:-3]


'2024-12-09-105-2'

In [251]:
# Extract content from the loaded documents
system_prompts = []
for doc in data:  # Iterate through the list of lists
    system_prompts.append({
            "title": doc['name'][:-3],  # Extract the filename from the source path
            "content": doc['content']  # Append the page_content to the system_prompts list
        })

In [252]:
system_prompts


[{'title': '2024-12-09-105-2',
  'content': '# Purpose\n- 당신은 사용자와 게임을 진행하는 수학 선생님 역할을 수행합니다.\n- 당신은 한국 아이들이 좋아하는 방식으로 수식을 설명합니다.\n- 다음 규칙을 따르는 수학 게임을 진행해주세요.\n \n<Game>\n## Game Rules\n- 각 주제는 5개의 단계로 구성되어 있으며, 주어진 힌트를 바탕으로 정확한 숫자를 입력해야 합니다.\n- 모든 문제는 덧셈, 뺄셈, 곱셈, 나눗셈의 혼합 계산으로 구성됩니다.\n- 모든 연산은 반드시 PEMDAS 순서를 따릅니다.\n- 오직 숫자만 정답으로 인정됩니다.\n \n### Game answer & question\n- 정답과 문제는 다음과 같습니다.\n- 문제는 5개 중 하나를 선택해 제시합니다.\n- 문제와 정답의 elements number는 동일합니다.\n<question_answer_pair>\nquestion_list = {question_list}\nanswer_list = {answer_list}\n</question_answer_pair>\n\n## Game Process\n1. 게임 시작 시 total_round는 1로 시작합니다.\n    - 게임 시작시 다음 문구와 함께 게임을 시작합니다:\n<start_message>\n"안녕하세요! 우리 주변의 다양한 것들 속에 숨어있는 재미있는 수학 문제를 찾아볼까요? ✨"\n</start_message>\n    - 게임 시작과 함께 문제와 첫 번째 힌트를 제시합니다.\n \n2. 매 라운드마다 다음을 진행합니다:\n    - question_answer_pair 에서 적합한 문제와 정답 1쌍을 고릅니다.\n    - 이 중 정답은 current_answer, 질문은 message의 value값으로 지정합니다.\n    - hint에 해당 문제의 풀이에 대한 힌트를 1개 제시합니다.\n3. 사용자 응답에 따른 처리:\n    - 정답인 경우:\n        -

In [253]:
system_prompts[18]['content']


'# Purpose\n- 당신은 사용자와 게임을 진행하는 영어 선생님 역할을 수행합니다.\n- 다음 규칙을 따르는 영어게임을 진행해주세요.\n<Game>\n## Game Overview\n- 플레이어는 5개 단계로 구성된 이루어진 가상의 공간에 있으며, 각 상황을 탈출하기 위해서는 주어진 힌트를 바탕으로 영어 단어를 정확한 스펠링으로 입력해야 합니다.\n### Game answer & question\n- 정답과 문제는 다음과 같습니다.\n- 문제는 5개 중 하나를 선택해 제시합니다.\n- 문제와 정답의 elements number는 동일합니다.\n<question_answer_pair> \nquestion_list = {question_list} \nanswer_list = {answer_list} \n</question_answer_pair>\n## Game Process\n1. 게임 시작 시 total_round는 1로 시작합니다.\n    - 게임 시작시 다음 문구와 함께 게임을 시작합니다:\n<start_message>\n    "동물 친구들의 달리기 대회가 시작되었어요!🐾🦁🦓 각 트랙에서 더 멀리간 동물의 이름을 영어로 입력해 보세요!🎯"\n</start_message>\n    - 게임 시작과 함께 문제를 제시합니다.\n2. 매 라운드마다 다음을 진행합니다:\n    - question_answer_pair 에서 적합한 문제와 정답 1쌍을 순차적으로 고릅니다.\n    - 이 중 정답은 current_answer, 질문은 message의 value값으로 지정합니다.\n    - hint에 해당 단어에 대한 힌트를 1개 제시합니다.\n3. 사용자 응답에 따른 처리:\n    - 정답인 경우:\n        - check_answer를 True로 설정\n        - answer_count를 1 증가\n        - 다음 라운드로 진행\n    - 오답인 경우:\n        - check_answer를 False로 설정\n        -

In [254]:
if system_prompts:
    system_prompt = system_prompts[0]['content']  # You can modify this to select a different prompt

    # Set up the chat model without system_prompt in model_kwargs
    chat = ChatOpenAI(model="gpt-4o-mini").bind(logprobs=True)

    # Example of how to use the chat model
    messages = [
        {"role": "system", "content": system_prompt},  # Include system prompt in messages
        {"role": "user", "content": "What is the first question?"}
    ]
    
    response = chat.invoke(messages)  # Invoke the chat model with the messages
    print(response)  # Print the response from the chat model

content='{\n    "total_round": 1,\n    "answer_count": 0,\n    "current_answer": "12",\n    "hint": [\n        "먼저, 3과 9를 더해보세요. 😊",\n        "그 다음, 그 결과에서 3을 빼보세요."\n    ],\n    "check_answer": false,\n    "is_end": false,\n    "message": "안녕하세요! 우리 주변의 다양한 것들 속에 숨어있는 재미있는 수학 문제를 찾아볼까요? ✨\\n문제를 풀어볼까요? 3 + 9 - 3의 결과는 무엇일까요?"\n}' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 136, 'prompt_tokens': 1354, 'total_tokens': 1490, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_6fc10e10eb', 'finish_reason': 'stop', 'logprobs': {'content': [{'token': '{\n', 'bytes': [123, 10], 'logprob': -0.10082311, 'top_logprobs': []}, {'token': '   ', 'bytes': [32, 32, 32], 'logprob': -2.1769476e-05, 'top_logprobs': []}, {'token': ' "', 'bytes

In [255]:
# Assuming 'response' is the AIMessage object
logprobs_content = response.response_metadata['logprobs']['content']

# Extract logprobs starting from "message"
logprobs_starting_from_message = []
found_message = False

for logprob in logprobs_content:
    if found_message:
        logprobs_starting_from_message.append(logprob)
    if logprob['token'] == 'message':
        found_message = True
        logprobs_starting_from_message.append(logprob)  # Include the "message" logprob itself

# Display the logprobs starting from "message"
for logprob in logprobs_starting_from_message:
    print(logprob)

{'token': 'message', 'bytes': [109, 101, 115, 115, 97, 103, 101], 'logprob': 0.0, 'top_logprobs': []}
{'token': '":', 'bytes': [34, 58], 'logprob': -3.1281633e-07, 'top_logprobs': []}
{'token': ' "', 'bytes': [32, 34], 'logprob': -2.8160932e-06, 'top_logprobs': []}
{'token': '안', 'bytes': [236, 149, 136], 'logprob': -6.704273e-07, 'top_logprobs': []}
{'token': '녕하세요', 'bytes': [235, 133, 149, 237, 149, 152, 236, 132, 184, 236, 154, 148], 'logprob': -9.014684e-06, 'top_logprobs': []}
{'token': '!', 'bytes': [33], 'logprob': 0.0, 'top_logprobs': []}
{'token': ' 우리', 'bytes': [32, 236, 154, 176, 235, 166, 172], 'logprob': -4.8425554e-06, 'top_logprobs': []}
{'token': ' 주변', 'bytes': [32, 236, 163, 188, 235, 179, 128], 'logprob': -1.2233183e-05, 'top_logprobs': []}
{'token': '의', 'bytes': [236, 157, 152], 'logprob': -1.9361265e-07, 'top_logprobs': []}
{'token': ' 다양한', 'bytes': [32, 235, 139, 164, 236, 150, 145, 237, 149, 156], 'logprob': -2.6968896e-06, 'top_logprobs': []}
{'token': ' 것',

In [256]:
scenario_list = ["인사/소개",
        "숫자/시간",
        "색상/모양",
        "기본 동작",
        "교실 물건",
        "학교 활동",
        "기본 지시어",
        "학습 관련",
        "가족/친구",
        "음식/음료",
        "취미/운동",
        "날씨/계절",
        "감정 표현",
        "의견 말하기",
        "선호도",
        "제안/요청",
        "장소/방향",
        "쇼핑/여행",
        "건강/병원",
        "직업/미래",
        "나라/문화",
        "환경/자연",
        "과학/기술",
        "예술/취미"]


In [257]:
answer_list = [[
            "hi",
            "name",
            "bye"
        ],
        [
            "one",
            "time",
            "six"
        ],
        [
            "red",
            "blue",
            "round"
        ],
        [
            "walk",
            "run",
            "jump"
        ],
        [
            "desk",
            "book",
            "pen"
        ],
        [
            "read",
            "write",
            "study"
        ],
        [
            "open",
            "close",
            "sit"
        ],
        [
            "test",
            "quiz",
            "score"
        ],
        [
            "mother",
            "father",
            "friend"
        ],
        [
            "water",
            "juice",
            "bread"
        ],
        [
            "swim",
            "play",
            "dance"
        ],
        [
            "spring",
            "summer",
            "rain"
        ],
        [
            "happy",
            "sad",
            "tired"
        ],
        [
            "think",
            "feel",
            "believe"
        ],
        [
            "best",
            "better",
            "favorite"
        ],
        [
            "help",
            "please",
            "could"
        ],
        [
            "library",
            "school",
            "right"
        ],
        [
            "store",
            "market",
            "shop"
        ],
        [
            "healthy",
            "doctor",
            "sick"
        ],
        [
            "dream",
            "hope",
            "wish"
        ],
        [
            "nature",
            "earth",
            "clean"
        ],
        [
            "science",
            "computer"
        ],
        [
            "robot",
            "phone",
            "web"
        ],
        [
            "paint",
            "draw",
            "sing"
        ]]


In [258]:
input_data = {
       "scenario_list": scenario_list,
       "answer_list": answer_list
   }

In [259]:
system_prompts[-3]


{'title': '아에이오우_확장 copy with_vars',
 'content': '# Purpose\n- 당신은 사용자와 함께 **영어 단어 맞히기 게임**을 진행하는 영어 선생님 역할을 수행합니다.\n- 아래 <Game> 섹션에 적힌 **모든 규칙**을 준수해야 합니다. \n- **규칙을 위반할 경우, 게임이 정상적으로 진행되지 않습니다.**\n\n<Game>\n## Game Overview\n- 사용자는 **5개 단계**로 이루어진 가상의 공간에 있고, 각 단계에서 영어 단어 문제를 푸는 식으로 **탈출**합니다.\n- 각 문제는 **Scenario List(시나리오)**와 **Answer List(정답 단어)**에서 1쌍을 무작위로 선택해 출제합니다.\n- 문제 문장 안에 정답 단어의 **모음 부분만 빈칸**(`_`)으로 표시합니다. \n  - (예: `b__k` → `book`)\n\n### Game topic & words\n- 시나리오와 단어가 순서대로 1:1 대응된다고 가정합니다.\n  - 예: `<scenario_list>\nscenario_list = {scenario_list}\nanswer_list = {answer_list}\n</scenario_list>`\n\n## Game Process\n\n1. **게임 시작**  \n   - `total_round`를 **1**로 초기화합니다.  \n   - `answer_count`를 **0**으로 초기화합니다.\n\n2. **매 라운드 진행**  \n   - `scenario_list`와 `answer_list` 중 **아직 출제되지 않은** 1쌍을 무작위로 선택합니다.  \n     - `current_answer`는 `answer_list`에서 뽑은 단어가 됩니다.  \n   - **새로운 문맥**의 한국어 문장을 생성하고, 그 문장 안에 `current_answer`의 **모음**만 빈칸(`_`) 처리하여 문제를 완성합니다.  \n     - 이전 라운드에서 사용한 문맥과 **동일

In [260]:
standard_prompt = PromptTemplate(template=system_prompts[-3]['content'])


In [261]:
standard_chain = standard_prompt | chat


In [262]:
standard_response = standard_chain.invoke(input_data).content


In [265]:
standard_response

'{\n    "total_round": 1,\n    "answer_count": 0,\n    "current_answer": "hi",\n    "hint": "친구를 만났을 때 주로 사용하는 인사말입니다.",\n    "check_answer": false,\n    "is_end": false,\n    "message": "다음 문제를 풀어보세요!"\n}'

In [266]:
from langchain.prompts import ChatPromptTemplate
